# Pulse programming and dynamical decoupling

Native gates is a set of carefully calibrated gates directly supported by the hardware. Behind the scene, native gates themselves are implemented by analog control signals, or "pulses", applying on the state of the qubits. Composing a program directly with pulse operations is called "pulse programming", giving you even more control than native gates. For example, you can quantum operations directly in pulses in order to implement error suppression schemes such as dynamical decoupling, or error mitigation methods such zero noise extrapolation. You can also improve the elementary operation by experimenting novel custom gate implementations.

In this notebook, we demonstrate the pulse programing features of AutoQASM with two examples, one with a qubit idling program and one with error suppression by dynamical decoupling sequences.

In [1]:
import numpy as np

import braket.experimental.autoqasm as aq
from braket.experimental.autoqasm import pulse
from braket.experimental.autoqasm.instructions import x, h, y, z, rx, rz, cz, measure

from braket.aws import AwsDevice
from braket.devices import Devices

# Rigetti Aspen-M-3 device is the targeted device for this example notebook
device = AwsDevice(Devices.Rigetti.AspenM3)

Unlike gate operations, pulse operations often target Frame objects. Basic pulse operations include setting the frequency and phase of a frame, playing a waveform, applying delay and capturing the output of a frame. The pulse operations are defined in the [pulse module](https://github.com/amazon-braket/amazon-braket-sdk-python/blob/feature/autoqasm/src/braket/experimental/autoqasm/pulse/pulse.py) of AutoQASM. You can read this [documentation](https://docs.aws.amazon.com/braket/latest/developerguide/braket-pulse.html) to learn more about frames and waveforms.

As a hello-world example, the pulse program below includes a `delay` instruction on the frame of the physical qubit `$0` followed by capture instruction on the same frame. The pulse program represents measuring a qubit after a variable idling duration. It can be used characterize the T1 coherence time by sampling the measurement results with different idling durations.  

In [2]:
@aq.main
def idle(qubit: int, idle_duration: float):
    control_frame = device.frames[f"q{qubit}_rf_frame"]
    pulse.delay(control_frame, idle_duration)
    pulse.capture_v0(control_frame)

Below prints out the OpenQASM script of the program. The pulse instructions, delay and capture, are in a `cal` block, which stand for "calibration". A `cal` block defines the scope for low level hardware instructions. Today, the hardware instructions are pulse instructions.

In [3]:
print(idle(0, 10e-6).to_ir())

OPENQASM 3.0;
defcalgrammar "openpulse";
cal {
    delay[10.0us] q0_rf_frame;
    capture_v0(q0_rf_frame);
}


## Error supression with dynamical decoupling sequences

In the `idle` program above, we intentionally add a delay instruction to the program. In more practical programs, qubit idling is inevitable. It often happens when a gate or pulse instruction applies on multiple qubits, but some of the qubit are still under other instructions. The rest of the qubits are idling and waiting for all qubits to be ready. During this idling time, the qubits are subjected to decoherence, resulting in low program fidelity. Dynamical decoupling is a technique to suppress such errors. It includes alternating pairs of X and Y "$\pi$ pulse", a rotation around the Bloch sphere axis with an angle $\pi$, applying on the qubits during the idling time. These pulses together are equivalent to an identity gate, but alternating "$\pi$ pulses" make some of the decoherence effectively cancel each other out. To learn more about dynamical decoupling, [this blog post](https://aws.amazon.com/blogs/quantum-computing/suppressing-errors-with-dynamical-decoupling-using-pulse-control-on-amazon-braket/) has a detailed explanations and visualization of dynamical decoupling sequences.

First, we create the X and Y $\pi$ pulse in terms of the native gates of Rigetti Aspen-M-3 device. 

In [4]:
pi = np.pi

def x_pulse(qubit: int):
    # Pi pulse apply on X-axis of Bloch sphere
    qubit = f"${qubit}"
    rx(qubit, pi)

def y_pulse(qubit: int):
    # Pi pulse apply on Y-axis of Bloch sphere
    qubit = f"${qubit}"
    rz(qubit, -0.5 * pi)
    rx(qubit, pi)
    rz(qubit, +0.5 * pi)

Each cycle of a dynamical decoupling is a sequence of X-Y-X-Y pulses, and each of the four pulses are equally distributed in a cycle. For example, if the idling duration assigned to a cycle is $8\tau$, each pulse is assigned to the middle of each $2\tau$ duration. The resulting pulse distribution of a dynamical decoupling cycle is shown in the figure below.

<img src="https://d2908q01vomqb2.cloudfront.net/5a5b0f9b7d3f8fc84c3cef8fd8efaaa6c70d75ab/2022/11/29/coldquanta_pulse_blog_pulses-2-1024x489.png" width="500">

The program below realizes this pattern of the dynamical decoupling sequence. The sequence is shown as a standalone program, but the same sequence can be inserted into any program that has idling qubits.

In [5]:
@aq.main
def idle_with_dd(qubit: int, idle_duration: float, n_cycles: int = 1) -> None:
    dd_spacing = idle_duration / (4*n_cycles)

    control_frame = device.frames[f"q{qubit}_rf_frame"]
    
    pulse.delay(control_frame, dd_spacing)
    for _ in aq.range(n_cycles):
        x_pulse(qubit)
        pulse.delay(control_frame, 2*dd_spacing)
        y_pulse(qubit)
        pulse.delay(control_frame, 2*dd_spacing)
        x_pulse(qubit)
        pulse.delay(control_frame, 2*dd_spacing)
        y_pulse(qubit)
        
    pulse.delay(control_frame, dd_spacing)

The OpenQASM script of the dynamical decoupling program is printed out below. 

In [6]:
print(idle_with_dd(0, 10e-6).to_ir())

OPENQASM 3.0;
defcalgrammar "openpulse";
cal {
    delay[2.5us] q0_rf_frame;
}
for int _ in [0:1 - 1] {
    rx(pi) $0;
    cal {
        delay[5.0us] q0_rf_frame;
    }
    rz(-(pi / 2)) $0;
    rx(pi) $0;
    rz(pi / 2) $0;
    cal {
        delay[5.0us] q0_rf_frame;
    }
    rx(pi) $0;
    cal {
        delay[5.0us] q0_rf_frame;
    }
    rz(-(pi / 2)) $0;
    rx(pi) $0;
    rz(pi / 2) $0;
}
cal {
    delay[2.5us] q0_rf_frame;
}


## Summary
This example shows you how to use pulse programming to create a program. Same as a gate-based circuit, a pulse program is compose in a function decorated by `@aq.main`. With AutoQASM, you can easily compose a pulse program because the pulse instructions are automatically added to `cal` blocks in OpenQASM script. There is no need to define additional scope that separates gate and pulse instructions in AutoQASM.  